
# CS5228 Project - Team not found




In [388]:
# import packages

import numpy as np
import pandas as pd

from src.utils import *

## Load Data

In [389]:

path = './'

original_train_df = pd.read_csv(path + 'data/train.csv')
original_test_df = pd.read_csv(path + 'data/test.csv')

In [390]:
# duplicate dataset

train_df = original_train_df.copy()
test_df = original_test_df.copy()

In [391]:
print(train_df.shape)
print(test_df.shape)

(26048, 23)
(7500, 22)


In [392]:
train_df.head()

,listing_id,name,street,type,model,market_segment,type_of_area,bedrooms,bathrooms,district,...,lat,lng,tenure,built_year,no_of_units,area_size,eco_category,accessibility,date_listed,price
0,6998418,seascape,57 cove way,condominium,condominium,ocr,strata,3,4.0,4,...,1.239337,103.837487,leasehold/99 years,2011.0,151.0,2336.0,uncategorized,guarded,2021-05-04,5390000.0
1,2046604,la maison,10 moulmein rise,apartment,apartment,ocr,strata,3,3.0,11,...,1.319533,103.847030,freehold,1999.0,24.0,1259.0,uncategorized,guarded,2021-05-07,2310000.0
2,7563513,viva,2 suffolk walk,condominium,condominium,ocr,strata,4,3.0,11,...,1.315656,103.844445,freehold,2012.0,235.0,1959.0,uncategorized,guarded,2021-06-30,5279500.0
3,3071445,urban treasures,205 jalan eunos,condominium,condominium,ocr,strata,3,2.0,14,...,1.329367,103.905791,freehold,NaN,237.0,883.0,uncategorized,guarded,2022-01-02,1843600.0
4,9667539,infini at east coast,east coast road,apartment,apartment,ocr,strata,3,3.0,15,...,1.309176,103.911352,freehold,NaN,36.0,1066.0,uncategorized,guarded,2021-12-24,2262700.0


In [393]:
print(train_df.dtypes)

listing_id          int64
name               object
street             object
type               object
model              object
market_segment     object
type_of_area       object
bedrooms           object
bathrooms         float64
district            int64
region             object
planning_area      object
subszone           object
lat               float64
lng               float64
tenure             object
built_year        float64
no_of_units       float64
area_size         float64
eco_category       object
accessibility      object
date_listed        object
price             float64
dtype: object


## Data Cleaning

### Check null value

In [394]:

# combine train and test 

train_test_df = pd.concat([train_df, test_df], axis=0)

compute_null_count(original_train_df)

listing_id            0
name                  0
street                0
type                  0
model              1280
market_segment        0
type_of_area          0
bedrooms            404
bathrooms          1236
district              0
region                0
planning_area         0
subszone              0
lat                   0
lng                   0
tenure              287
built_year        10043
no_of_units        1042
area_size             2
eco_category          0
accessibility         0
date_listed           0
price                 0
dtype: int64

#### model

In [395]:
# training 

print(compute_null_count(original_train_df, ["model"]))

# fill null with the same property name
train_df = fill_missing_value_by_group_neighbor(train_df, 'model', 'name')
print(compute_null_count(train_df, ["model"]))

# fill null by mode
train_df, mode = fill_missing_value_by_mode(train_df, "model")
print(compute_null_count(train_df, ["model"]))


model    1280
dtype: int64
model    24
dtype: int64
model    0
dtype: int64


In [396]:
# testing
print(compute_null_count(test_df, ["model"]))

test_df = fill_missing_value_by_group_neighbor(test_df, "model", "name")
print(compute_null_count(test_df, ["model"]))

test_df, mode = fill_missing_value_by_mode(test_df, 'model')
print(compute_null_count(test_df, ["model"]))

model    388
dtype: int64
model    20
dtype: int64
model    0
dtype: int64


#### area size

In [397]:
# area_size

# fill null by median
train_df.loc[train_df['area_size'].isnull(), 'area_size'] = train_df['area_size'].median()

In [398]:
# test code

df = pd.DataFrame([['a', 6.2,  1], ['b', 0., None], ['b', 1.5, 6], ['c', 1.6, 7], ['a',2., None]])

df.columns = ['k', 'g', 'value']

display(df)

df = fill_missing_value_by_group_neighbor(df, 'value', 'k')

display(df)

,k,g,value
0,a,6.2,1.0
1,b,0.0,NaN
2,b,1.5,6.0
3,c,1.6,7.0
4,a,2.0,NaN


,k,g,value
0,a,6.2,1.0
1,b,0.0,6.0
2,b,1.5,6.0
3,c,1.6,7.0
4,a,2.0,1.0


#### bedroom

In [399]:
# training

# break into bedrooms and additional rooms
print(compute_null_count(train_df, ['bedrooms']))

# fill null according to area size
train_df = fill_missing_value_by_neighbor(train_df, 'bedrooms', 'area_size')

print(compute_null_count(train_df, ['bedrooms']))


# split to two columns
train_df[['bedrooms', 'additional_rooms']] = train_df['bedrooms'].str.split('+', expand=True).fillna(0)
train_df[['bedrooms', 'additional_rooms']] = train_df[['bedrooms', 'additional_rooms']].astype('int')

bedrooms    404
dtype: int64
bedrooms    0
dtype: int64


In [400]:
# testing

print(compute_null_count(test_df, ['bedrooms']))

# fill null according to area size
test_df = fill_missing_value_by_neighbor(test_df, 'bedrooms', 'area_size')
# print(compute_null_count(test_df, ['bedrooms']))

test_df[['bedrooms']] = test_df[['bedrooms']].fillna(1)

# split to to columns
test_df[['bedrooms', 'additional_rooms']] = test_df['bedrooms'].str.split('+', expand=True)
test_df[['bedrooms', 'additional_rooms']] = test_df[['bedrooms', 'additional_rooms']].replace(r'', np.nan)
test_df[['bedrooms', 'additional_rooms']] = test_df[['bedrooms', 'additional_rooms']].fillna(0)
test_df[['bedrooms', 'additional_rooms']] = test_df[['bedrooms', 'additional_rooms']].astype('int')

print(compute_null_count(test_df, ['bedrooms', 'additional_rooms']))

bedrooms    118
dtype: int64
bedrooms            0
additional_rooms    0
dtype: int64


#### bathroom


In [401]:
# training
print(compute_null_count(train_df, ['bathrooms']))

# fill null by bedrooms
train_df = fill_missing_value_by_neighbor(train_df, 'bathrooms', 'bedrooms')
print(compute_null_count(train_df, ['bathrooms']))

bathrooms    1236
dtype: int64
bathrooms    0
dtype: int64


In [402]:
# testing
print(compute_null_count(test_df, ['bathrooms']))

test_df = fill_missing_value_by_neighbor(test_df, 'bathrooms', 'bedrooms')
print(compute_null_count(test_df, ['bathrooms']))


bathrooms    337
dtype: int64
bathrooms    0
dtype: int64


#### tenure

https://www.propertyguru.com.sg/property-guides/singapore-freehold-vs-leasehold-which-is-better-9520

In [ ]:
# training

# fill null by name 
print(compute_null_count(train_df, ['tenure']))
train_df = fill_missing_value_by_group_neighbor(train_df, 'tenure', 'name')
print(compute_null_count(train_df, ['tenure']))

# fill null by street
train_df = fill_missing_value_by_group_neighbor(train_df, 'tenure', 'street')
print(compute_null_count(train_df, ['tenure']))

# by mode 
train_df['tenure'] = train_df['tenure'].fillna(train_df[['tenure']].mode().iloc[0, 0])

# convert to binary columns freehold



tenure    287
dtype: int64


In [ ]:

# change freehold to 999
train_df.loc[train_df['tenure'] == 'freehold', 'tenure'] = '999 years'


freehold = train_df['tenure'].str.extract('(\d+)').astype('int')

freehold[freehold <= 99] = 0
freehold[freehold > 99] = 1

train_df = train_df.rename(columns={'tenure': 'freehold'})
train_df['freehold'] = freehold


In [ ]:
# testing
# fill null by name
print(compute_null_count(test_df, ['tenure']))
test_df = fill_missing_value_by_group_neighbor(test_df, 'tenure', 'name')
print(compute_null_count(test_df, ['tenure']))

# by street
test_df = fill_missing_value_by_group_neighbor(test_df, 'tenure', 'street')
print(compute_null_count(test_df, ['tenure']))

# by mode
test_df['tenure'] = test_df['tenure'].fillna(train_test_df[['tenure']].mode().iloc[0, 0])
print(compute_null_count(test_df, ['tenure']))

In [ ]:

# change freehold to 999
test_df.loc[test_df['tenure'] == 'freehold', 'tenure'] = '999 years'

freehold = test_df['tenure'].str.extract('(\d+)').astype('int')


freehold[freehold <= 99] = 0
freehold[freehold > 99] = 1

test_df = test_df.rename(columns={'tenure': 'freehold'})
test_df['freehold'] = freehold


test_df[['freehold']].value_counts()

#### built year

In [ ]:
# training

# fill null 
print(compute_null_count(train_df, ['built_year']))

# by name
train_df = fill_missing_value_by_group_neighbor(train_df, 'built_year', 'name')
print(compute_null_count(train_df, ['built_year']))

# filled by 'unknown'
train_df['built_year'] = train_df['built_year'].fillna('unknown')
print(compute_null_count(train_df, ['built_year']))

In [ ]:
# testing

# fill null 
print(compute_null_count(test_df, ['built_year']))

# by name
test_df = fill_missing_value_by_group_neighbor(test_df, 'built_year', 'name')
print(compute_null_count(test_df, ['built_year']))

# filled by 'unknown'
test_df['built_year'] = test_df['built_year'].fillna('unknown')
print(compute_null_count(test_df, ['built_year']))

#### no_of_units

In [ ]:
# training
print(compute_null_count(train_df, ['no_of_units']))


# by name
train_df = fill_missing_value_by_group_neighbor(train_df, 'no_of_units', 'name')
print(compute_null_count(train_df, ['no_of_units']))


# by mean
train_df['no_of_units'] = train_df['no_of_units'].fillna(train_df['no_of_units'].mean())
print(compute_null_count(train_df, ['no_of_units']))



In [ ]:
# testing
print(compute_null_count(test_df, ['no_of_units']))


# by name
test_df = fill_missing_value_by_group_neighbor(test_df, 'no_of_units', 'name')
print(compute_null_count(test_df, ['no_of_units']))


# by mean
test_df['no_of_units'] = test_df['no_of_units'].fillna(train_test_df['no_of_units'].mean())
print(compute_null_count(test_df, ['no_of_units']))




#### date_listed

In [ ]:
train_df['date_listed'] = pd.to_datetime(train_df['date_listed'])
test_df['date_listed'] = pd.to_datetime(test_df['date_listed'])

In [ ]:
# check null
compute_null_count(train_df)

In [ ]:
compute_null_count(test_df)

### Check duplicates

In [ ]:
print(np.sum(train_df['listing_id'].duplicated()))
print(np.sum(test_df['listing_id'].duplicated()))

### Drop useless columns



In [ ]:
print(train_df['market_segment'].value_counts())
print(train_df['type_of_area'].value_counts())
print(train_df['eco_category'].value_counts())
print(train_df['accessibility'].value_counts())

In [ ]:
print(test_df['market_segment'].value_counts())
print(test_df['type_of_area'].value_counts())
print(test_df['eco_category'].value_counts())
print(test_df['accessibility'].value_counts())

In [ ]:
# remove only one values and too many categories

to_drop_columns = ['market_segment', 'type_of_area', 'eco_category', 'accessibility'] + \
                    ['listing_id', 'name', 'street', 'subszone']


In [ ]:
train_df = train_df.drop(columns=to_drop_columns)
train_df.shape

In [ ]:
test_df = test_df.drop(columns=to_drop_columns)
test_df.shape

In [ ]:
# split categorical and numerical features 
cat_feature_names = [train_df.dtypes.index[i] for i in range(train_df.shape[1]) if train_df.dtypes.values[i] == object]
num_feature_names = [train_df.dtypes.index[i] for i in range(train_df.shape[1]) if train_df.dtypes.values[i] != object]

### Categorical features